### Importing required libraries

In [219]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

### Creating dataframe to store required data

In [220]:
df1_config=pd.read_excel(r"C:\Users\Naina Sisodia\Desktop\excel_to_python.xlsx", sheet_name='config_data')
df2_proc_data=pd.read_excel(r"C:\Users\Naina Sisodia\Desktop\excel_to_python.xlsx", sheet_name='Data')
df_tag_list=pd.read_excel(r"C:\Users\Naina Sisodia\Desktop\excel_to_python.xlsx", sheet_name='tag_description')
df_final_cols=pd.read_excel(r"C:\Users\Naina Sisodia\Desktop\excel_to_python.xlsx", sheet_name='final_col_list')
df_coeff_summary=pd.read_excel(r"C:\Users\Naina Sisodia\Desktop\GERC_A_PARA_GUI_rev4.xlsm", sheet_name='Coeff_Summary')

In [221]:
df1_config.head() #this is how our dataframe would look df1 to df4

,tag_name,stg1_values,stg2_values
0,reference_pressure,1.01325,NaN
1,reference_temperature,15.60000,NaN
2,suction_temperature,54.40000,55.00000
3,stg_inlet_capacity_volumetric_flow,2834.90000,34143.20689
4,cylinder_dia,330.20000,241.30000


In [222]:
df2_proc_data.head(3)

,Date,Time,GERC_A_TIT_003.PV,GERC_A_TIT_004.PV,GERC_A_TIT_010.PV,GERC_A_TIT_006.PV,GERC_A_TIT_007.PV,GERC_A_TIT_020.PV,GERC_A_TIT_015.PV,GERC_A_TIT_016.PV,...,GERC_A_VE_VT_016.PV,GERC_A_VE_VT_017.PV,GERC_A_VE_VT_018.PV,GERC_A_VE_VT_019.PV,GERC_A_VE_VT_020.PV,GERC_A_VE_VT_015A.PV,GERC_A_VE_VT_015B.PV,GERC_A_VT_001A.PV,GERC_A_VT_002A.PV,GERC_A_VT_003A.PV
0,2019-01-14,18:00:00,23.64,51.70,21.55,21.82,51.23,49.82,52.03,51.76,...,1.06,0.97,1.46,1.28,1.83,1.64,1.44,1.83,1.46,1.74
1,2019-01-14,19:00:00,23.96,55.85,21.20,21.40,56.07,53.12,55.91,55.76,...,1.39,1.24,1.96,1.70,2.41,2.17,1.91,1.84,1.47,1.80
2,2019-01-14,20:00:00,24.29,59.99,20.85,20.99,60.91,56.41,59.80,59.77,...,1.72,1.52,2.45,2.13,2.98,2.70,2.37,1.85,1.48,1.87


In [223]:
df_tag_list.head()

,client_tag_name,description,org_description
0,GERC_A_TIT_003.PV,S1_suction_temp,"1st Stage Suction, deg C"
1,GERC_A_TIT_004.PV,S1_discharge_temp,"1st Stage Discharge, deg C"
2,GERC_A_TIT_010.PV,HE_outlet,Air Cooled Heat Exchanger Outlet
3,GERC_A_TIT_006.PV,S2_suction_srubber_out,2nd Stage Suction Scrubber Outlet
4,GERC_A_TIT_007.PV,S2_discharge_temp,2nd Stage Discharge


In [224]:
df_final_cols.head()

,final_col_list,new_name,Unnamed: 2
0,Suction pressure @ flange,suction_pressure_flange,NaN
1,Discharge pressure @ flange,discharge_pressure_flange,NaN
2,Compression ratio,compression_rartio,NaN
3,Total Swept volume,total_swept_vol,NaN
4,"Swept vol, HE",swept_vol_HE,NaN


### Mapping our client's tag list to the new column name as per our convenience

In [225]:
col_mapping=dict(zip(df_tag_list['client_tag_name'],df_tag_list['description']))#this will help us to use our column name for calculation irrespective of client's tag_name
df2_proc_data.rename(columns=col_mapping, inplace=True) #this will rename client_tag with our column_name, which we specified in df3

In [226]:
df2_proc_data.head(2)

,Date,Time,S1_suction_temp,S1_discharge_temp,HE_outlet,S2_suction_srubber_out,S2_discharge_temp,inlet_to_oilfilter,compressor_main_bearingrtd,compressor_main_bearingrtd,...,compressorframe,compressorframe,compressorframe,compressorframe,enginevibration,CP_vibration_pressure,CP_vibration_pressure,HE_motor,HE_motor,HE_motor
0,2019-01-14,18:00:00,23.64,51.70,21.55,21.82,51.23,49.82,52.03,51.76,...,1.06,0.97,1.46,1.28,1.83,1.64,1.44,1.83,1.46,1.74
1,2019-01-14,19:00:00,23.96,55.85,21.20,21.40,56.07,53.12,55.91,55.76,...,1.39,1.24,1.96,1.70,2.41,2.17,1.91,1.84,1.47,1.80


### Making a list of final columns which we've to calculate

In [227]:
final_col_list=df_final_cols['new_name']

In [228]:
print(final_col_list)

0          suction_pressure_flange
1        discharge_pressure_flange
2               compression_rartio
3                  total_swept_vol
4                     swept_vol_HE
5                     swept_vol_CE
6                polymetric_expo_n
7              HE_clearance_vol_Vs
8                         (V4) TDC
9                      head_end_V1
10                          BDC_V2
11                           HE_V3
12             CE_clearance_vol_Vs
13                          TDC_V4
14                           CE_V1
15                          BDC_V2
16                           HE_V3
17              HE_inlet_vol_eff %
18              CE_inlet_vol_eff %
19          HE_discharge_vol_eff %
20          CE_discharge_vol_eff %
21          cal_avg_inlet_vol_eff%
22       design_avg_inlet_vol_eff%
23                overall_vol_eff%
24                           Zs_S1
25                           Zd_S1
26                            Zavg
27                    Vsp_inlet_S1
28                Vs

In [229]:
### Defining a function to store our formulae for final_cols

In [230]:
#df_res_stg1=calc(df1_config, df2_proc_data,'stg1')

In [231]:
stage=['STAGE I']
vsp_const=8314.4
time_conv=3600*1000

def calc(df1_config, df2_proc_data, stage):
    
    suction_col_name = 'suction_pressure_flange_' + str(stage)
    discharge_col_name = 'discharge_pressure_flange_' + str(stage)
    ratio_col_name = 'compression_ratio_' + str(stage)  # Corrected typo here from rartio to ratio
    tot_swept_col_name = 'total_swept_vol_' + str(stage)
    swept_HEvol_col_name = 'swept_vol_HE_' + str(stage)
    swept_CEvol_col_name = 'swept_vol_CE_' + str(stage)
    polymetric_expo_n_col_name = 'polymetric_expo_n_' + str(stage)
    HE_clearance_vol_Vs_col_name = 'HE_clearance_vol_Vs_' + str(stage)
    V4_TDC_col_name='(V4) TDC'+str(stage)
    head_end_V1_col_name='head_end_V1'+str(stage)
    BDC_V2_col_name='BDC_V2'+str(stage)
    HE_V3_col_name='HE_V3'+str(stage)
    CE_clearance_vol_Vs_col_name='CE_clearance_vol_Vs'+str(stage)
    TDC_V4_col_name='TDC_V4'+str(stage)
    CE_V1_col_name='CE_V1'+str(stage)
    BDC_V2_col_name2='BDC_V2_'+str(stage)
    HE_V3_col_name2='HE_V3(2)'+str(stage)
    HE_inlet_vol_eff_col_name='HE_inlet_vol_eff %'+str(stage)
    CE_inlet_vol_eff_col_name='CE_inlet_vol_eff %'+str(stage)
    HE_discharge_vol_eff_col_name='HE_discharge_vol_eff %'+str(stage)
    CE_discharge_vol_eff_col_name='CE_discharge_vol_eff %'+str(stage)
    cal_avg_inlet_vol_eff_col_name='cal_avg_inlet_vol_eff%'+str(stage)
    design_avg_inlet_vol_eff_col_name='design_avg_inlet_vol_eff%'+str(stage)
    overall_vol_eff_col_name='overall_vol_eff%'+str(stage)
    stage_inlet_capacity_Q_col_name='stage_inlet_capacity_Q'+str(stage)
    stage_discharge_capacity_Q_col_name='stage_discharge_capacity_Q'+str(stage)
    actual_disc_temp_S1_col_name='actual_disc_temp_S1'+str(stage)
    rod_load_tension_col_name='rod_load_tension'+str(stage)
    rod_load_compression_col_name='rod_load_compression'+str(stage)
    max_allowed_gas_load_col_name='max_allowed_gas_load'+str(stage)
    Zs_S1_col_name='Zs_S1'+str(stage)
    Zd_S1_col_name='Zd_S1'+str(stage)
    Zavg_col_name='Zavg'+str(stage)
    Vsp_inlet_S1_col_name='Vsp_inlet_S1'+str(stage)
    Vsp_discharge_S1_col_name='Vsp_discharge_S1'+str(stage)
    stage_inlet_capacity_M_col_name='stage_inlet_capacity_M'+str(stage)
    stage_discharge_capacity_M_col_name='stage_discharge_capacity_M'+str(stage)
    k_col_name='k'+str(stage)
    k_k1_col_name='k/k-1'+str(stage)
    Had_col_name='Had'+str(stage)
    Adiabatic_disc_temp_col_name='Adiabatic_disc_temp'+str(stage)
    Adiabatic_eff_col_name='Adiabatic_eff%'+str(stage)
    Pad_col_name='Pad'+str(stage)
    Zd_adiabatic_temp_disc_press_col_name='Zd_adiabatic_temp_disc_press'+str(stage)
    Vsp_discharge_col_name='Vsp_discharge'+str(stage)
    adiabatic_flow_factor_col_name='adiabatic_flow_factor'+str(stage)
    flow_factor_S1_col_name='flow_factor_S1'+str(stage)
    disc_temp_ad_disc_temp_col_name='disc_temp-ad_disc_temp'+str(stage)
    
    df2_proc_data[suction_col_name] = df2_proc_data['S1_suction_pressure'] + (df1_config.loc[df1_config['tag_name'] == 'atmospheric_pressure', 'stg1_values'].iloc[0])
    df2_proc_data[discharge_col_name] = df2_proc_data['S1_discharge_pressure'] + (df1_config.loc[df1_config['tag_name'] == 'dp_flange_discharge_line', 'stg1_values'].iloc[0]) + (df1_config.loc[df1_config['tag_name'] == 'atmospheric_pressure', 'stg1_values'].iloc[0])
    df2_proc_data[ratio_col_name] = df2_proc_data[discharge_col_name] / df2_proc_data[suction_col_name]
    df2_proc_data[tot_swept_col_name] = df1_config.loc[df1_config['tag_name'] == 'total_swept_volume', 'stg1_values'].iloc[0]
    df2_proc_data[swept_HEvol_col_name] = df1_config.loc[df1_config['tag_name'] == 'swept_volume_head_end', 'stg1_values'].iloc[0]
    df2_proc_data[swept_CEvol_col_name] = df1_config.loc[df1_config['tag_name'] == 'swept_volume_crank_end', 'stg1_values'].iloc[0]
    df2_proc_data[HE_clearance_vol_Vs_col_name] = ((df1_config.loc[df1_config['tag_name'] == 'head_end_clearance', 'stg1_values'].iloc[0])/100)*df2_proc_data[swept_HEvol_col_name]
    df2_proc_data[V4_TDC_col_name]=df2_proc_data[HE_clearance_vol_Vs_col_name]
    df2_proc_data[polymetric_expo_n_col_name] = 1 / (1 - ((np.log((df2_proc_data['S1_discharge_temp'] + 273) / (df2_proc_data['S1_suction_temp'] + 273))) / (np.log((df2_proc_data[discharge_col_name]/df2_proc_data[suction_col_name]))))) 
    df2_proc_data[head_end_V1_col_name]=df2_proc_data[V4_TDC_col_name]*((df2_proc_data[ratio_col_name])**(1/df2_proc_data[polymetric_expo_n_col_name]))
    df2_proc_data[BDC_V2_col_name]=df2_proc_data[swept_HEvol_col_name]+df2_proc_data[HE_clearance_vol_Vs_col_name]
    df2_proc_data[HE_V3_col_name]=df2_proc_data[BDC_V2_col_name]*((1/(df2_proc_data[ratio_col_name]))**(1/df2_proc_data[polymetric_expo_n_col_name]))
    df2_proc_data[CE_clearance_vol_Vs_col_name]=((df1_config.loc[df1_config['tag_name'] == 'crank_end_clearance', 'stg1_values'].iloc[0])/100)*df2_proc_data[swept_CEvol_col_name]
    df2_proc_data[TDC_V4_col_name]=df2_proc_data[CE_clearance_vol_Vs_col_name]
    df2_proc_data[CE_V1_col_name]=df2_proc_data[CE_clearance_vol_Vs_col_name]*((df2_proc_data[ratio_col_name])**(1/df2_proc_data[polymetric_expo_n_col_name]))
    df2_proc_data[BDC_V2_col_name2]=df2_proc_data[swept_CEvol_col_name]+df2_proc_data[CE_clearance_vol_Vs_col_name]
    df2_proc_data[HE_V3_col_name2]=df2_proc_data[BDC_V2_col_name2]*((1/(df2_proc_data[ratio_col_name]))**(1/df2_proc_data[polymetric_expo_n_col_name]))
    df2_proc_data[HE_inlet_vol_eff_col_name]=((df2_proc_data[BDC_V2_col_name]-df2_proc_data[head_end_V1_col_name])*100)/(df2_proc_data[BDC_V2_col_name]-df2_proc_data[V4_TDC_col_name])
    df2_proc_data[CE_inlet_vol_eff_col_name]=((df2_proc_data[BDC_V2_col_name2]-df2_proc_data[CE_V1_col_name])*100)/(df2_proc_data[BDC_V2_col_name2]-df2_proc_data[TDC_V4_col_name])
    df2_proc_data[HE_discharge_vol_eff_col_name]=((df2_proc_data[HE_V3_col_name]-df2_proc_data[V4_TDC_col_name])*100)/(df2_proc_data[BDC_V2_col_name]-df2_proc_data[V4_TDC_col_name])
    df2_proc_data[CE_discharge_vol_eff_col_name]=(((df2_proc_data[HE_V3_col_name2]-df2_proc_data[TDC_V4_col_name]))*100)/(df2_proc_data[BDC_V2_col_name2]-df2_proc_data[TDC_V4_col_name])
    df2_proc_data[cal_avg_inlet_vol_eff_col_name]=(df2_proc_data[HE_inlet_vol_eff_col_name]+df2_proc_data[CE_inlet_vol_eff_col_name])/2
    df2_proc_data[design_avg_inlet_vol_eff_col_name]=(df1_config.loc[df1_config['tag_name'] == 'volumetric_efficiency', 'stg1_values'].iloc[0])
    df2_proc_data[overall_vol_eff_col_name]=(df2_proc_data[HE_inlet_vol_eff_col_name]+df2_proc_data[CE_inlet_vol_eff_col_name]+df2_proc_data[HE_discharge_vol_eff_col_name]+df2_proc_data[CE_discharge_vol_eff_col_name])/4
    df2_proc_data[stage_inlet_capacity_Q_col_name]=df2_proc_data[cal_avg_inlet_vol_eff_col_name]*df2_proc_data[tot_swept_col_name]/100
    df2_proc_data[stage_discharge_capacity_Q_col_name]=(df2_proc_data[HE_discharge_vol_eff_col_name]+df2_proc_data[CE_discharge_vol_eff_col_name])/2*df2_proc_data[tot_swept_col_name]/100
    df2_proc_data[actual_disc_temp_S1_col_name]=df2_proc_data['S1_discharge_temp'] + 273
    df2_proc_data[rod_load_tension_col_name]=(((df1_config.loc[df1_config['tag_name'] == 'cross_sectional_area_of_piston', 'stg1_values'].iloc[0]) * (df2_proc_data[discharge_col_name] - df2_proc_data[suction_col_name])) - ((df1_config.loc[df1_config['tag_name'] == 'cross_sectional_area_of_piston_rod', 'stg1_values'].iloc[0]) * df2_proc_data[discharge_col_name])) * 10**5
    df2_proc_data[rod_load_compression_col_name]=(((df1_config.loc[df1_config['tag_name'] == 'cross_sectional_area_of_piston', 'stg1_values'].iloc[0]) * (df2_proc_data[discharge_col_name] - df2_proc_data[suction_col_name])) + ((df1_config.loc[df1_config['tag_name'] == 'cross_sectional_area_of_piston_rod', 'stg1_values'].iloc[0]) * df2_proc_data[suction_col_name])) * 10**5
    df2_proc_data[max_allowed_gas_load_col_name]=df1_config.loc[df1_config['tag_name'] == 'max_allowed_gas_rod_load', 'stg1_values'].iloc[0]
    df2_proc_data[Zs_S1_col_name]=df_coeff_summary.iloc[8, 1] + (df_coeff_summary.iloc[8, 3] * df2_proc_data['S1_suction_temp'])+ (df_coeff_summary.iloc[8, 4] * df2_proc_data[suction_col_name] * 100) + (df_coeff_summary.iloc[8, 5]* ((df2_proc_data['S1_suction_temp'] ** 2))) + (df_coeff_summary.iloc[8, 6] * df2_proc_data['S1_suction_temp'] * df2_proc_data[suction_col_name] * 100) + (df_coeff_summary.iloc[8, 7] * ((df2_proc_data[suction_col_name] * 100) ** 2))
    df2_proc_data[Zd_S1_col_name]=df_coeff_summary.iloc[6, 1] + (df_coeff_summary.iloc[6, 3] * df2_proc_data['S1_discharge_temp'])+ (df_coeff_summary.iloc[6, 4] * df2_proc_data[discharge_col_name] * 100) + (df_coeff_summary.iloc[6, 5]* ((df2_proc_data['S1_discharge_temp'] ** 2))) + (df_coeff_summary.iloc[6, 6] * df2_proc_data['S1_discharge_temp'] * df2_proc_data[discharge_col_name] * 100) + (df_coeff_summary.iloc[6, 7] * ((df2_proc_data[discharge_col_name] * 100) ** 2))
    df2_proc_data[Zavg_col_name]=(df2_proc_data[Zs_S1_col_name]+df2_proc_data[Zd_S1_col_name])/2
    df2_proc_data[Vsp_inlet_S1_col_name]=df2_proc_data[Zs_S1_col_name]*vsp_const*(df2_proc_data['S1_suction_temp'] + 273)/((df1_config.loc[df1_config['tag_name'] == 'mol_wt', 'stg1_values'].iloc[0])*df2_proc_data[suction_col_name]*(10**5))
    df2_proc_data[Vsp_discharge_S1_col_name]=df2_proc_data[Zd_S1_col_name]*vsp_const*(df2_proc_data['S1_discharge_temp'] + 273)/((df1_config.loc[df1_config['tag_name'] == 'mol_wt', 'stg1_values'].iloc[0])*df2_proc_data[discharge_col_name]*(10**5))
    df2_proc_data[stage_inlet_capacity_M_col_name]=df2_proc_data[stage_inlet_capacity_Q_col_name]/df2_proc_data[Vsp_inlet_S1_col_name]
    df2_proc_data[stage_discharge_capacity_M_col_name]=df2_proc_data[stage_discharge_capacity_Q_col_name]/df2_proc_data[Vsp_discharge_S1_col_name]
    df2_proc_data[k_col_name]=((df_coeff_summary.iloc[2, 1] + (df_coeff_summary.iloc[2, 3] * df2_proc_data['S1_suction_temp'])+ (df_coeff_summary.iloc[2, 4] * df2_proc_data[suction_col_name] * 100) + (df_coeff_summary.iloc[2, 5]* ((df2_proc_data['S1_suction_temp'] ** 2))) + (df_coeff_summary.iloc[2, 6] * df2_proc_data['S1_suction_temp'] * df2_proc_data[suction_col_name] * 100) + (df_coeff_summary.iloc[2, 7] * ((df2_proc_data[suction_col_name] * 100) ** 2)))+(df_coeff_summary.iloc[0, 1] + (df_coeff_summary.iloc[0, 3] * df2_proc_data['S1_discharge_temp'])+ (df_coeff_summary.iloc[0, 4] * df2_proc_data[discharge_col_name] * 100) + (df_coeff_summary.iloc[0, 5]* ((df2_proc_data['S1_discharge_temp'] ** 2))) + (df_coeff_summary.iloc[0, 6] * df2_proc_data['S1_discharge_temp'] * df2_proc_data[discharge_col_name] * 100) + (df_coeff_summary.iloc[0, 7] * ((df2_proc_data[discharge_col_name] * 100) ** 2))))/2
    df2_proc_data[k_k1_col_name]=df2_proc_data[k_col_name]/(df2_proc_data[k_col_name]-1)
    df2_proc_data[Had_col_name]=df2_proc_data[Zavg_col_name]*(vsp_const/(df1_config.loc[df1_config['tag_name'] == 'mol_wt', 'stg1_values'].iloc[0]))*(df2_proc_data['S1_suction_temp'] + 273)*df2_proc_data[k_k1_col_name]*(df2_proc_data[ratio_col_name]**(1/df2_proc_data[k_k1_col_name])-1)
    df2_proc_data[Adiabatic_disc_temp_col_name]=(df2_proc_data['S1_suction_temp'] + 273)*(df2_proc_data[ratio_col_name]**(1/df2_proc_data[k_k1_col_name]))
    df2_proc_data[Adiabatic_eff_col_name]=(df2_proc_data[Adiabatic_disc_temp_col_name]-((df2_proc_data['S1_suction_temp'] + 273)))*100/((df2_proc_data['S1_discharge_temp'])-(df2_proc_data['S1_suction_temp']))
    df2_proc_data[Pad_col_name]=df2_proc_data[stage_inlet_capacity_M_col_name]*df2_proc_data[Had_col_name]/(df2_proc_data[Adiabatic_eff_col_name]/100)*(1/time_conv)
    df2_proc_data[Zd_adiabatic_temp_disc_press_col_name]=df_coeff_summary.iloc[6, 1] + (df_coeff_summary.iloc[6, 3] * (df2_proc_data[Adiabatic_disc_temp_col_name]-273))+ (df_coeff_summary.iloc[6, 4] * df2_proc_data[discharge_col_name] * 100) + (df_coeff_summary.iloc[6, 5]* (((df2_proc_data[Adiabatic_disc_temp_col_name]-273) ** 2))) + (df_coeff_summary.iloc[6, 6] * (df2_proc_data[Adiabatic_disc_temp_col_name]-273) * df2_proc_data[discharge_col_name] * 100) + (df_coeff_summary.iloc[6, 7] * ((df2_proc_data[discharge_col_name] * 100) ** 2))
    df2_proc_data[Vsp_discharge_col_name]=df2_proc_data[Zd_adiabatic_temp_disc_press_col_name]*(vsp_const/(df1_config.loc[df1_config['tag_name'] == 'mol_wt', 'stg1_values'].iloc[0]))*(df2_proc_data[Adiabatic_disc_temp_col_name]/(df2_proc_data[suction_col_name]*(10**5)))
    df2_proc_data[adiabatic_flow_factor_col_name]=df2_proc_data[stage_inlet_capacity_M_col_name]/(df2_proc_data[stage_inlet_capacity_Q_col_name]/df2_proc_data[Vsp_discharge_col_name])
    df2_proc_data[flow_factor_S1_col_name]=df2_proc_data[stage_inlet_capacity_M_col_name]/df2_proc_data[stage_discharge_capacity_M_col_name]
    df2_proc_data[disc_temp_ad_disc_temp_col_name]=df2_proc_data[actual_disc_temp_S1_col_name]-df2_proc_data[Adiabatic_disc_temp_col_name]


calc(df1_config, df2_proc_data, stage) 

print(df2_proc_data.head())

        Date      Time  S1_suction_temp  S1_discharge_temp  HE_outlet  \
0 2019-01-14  18:00:00            23.64              51.70      21.55   
1 2019-01-14  19:00:00            23.96              55.85      21.20   
2 2019-01-14  20:00:00            24.29              59.99      20.85   
3 2019-01-14  21:00:00            24.62              64.14      20.50   
4 2019-01-14  22:00:00            24.95              68.29      20.15   

   S2_suction_srubber_out  S2_discharge_temp  inlet_to_oilfilter  \
0                   21.82              51.23               49.82   
1                   21.40              56.07               53.12   
2                   20.99              60.91               56.41   
3                   20.57              65.75               59.71   
4                   20.15              70.59               63.00   

   compressor_main_bearingrtd  compressor_main_bearingrtd  ...  \
0                       52.03                       51.76  ...   
1                   

In [232]:
vsp_const=8314.4
time_conv=3600*1000

def calc(df1_config, df2_proc_data, stage):
    
    suction_col_name2 = 'suction_pressure_flange_' + 'STAGE II'
    discharge_col_name2 = 'discharge_pressure_flange_' + 'STAGE II'
    ratio_col_name2 = 'compression_ratio_' + 'STAGE II'  # Corrected typo here from rartio to ratio
    tot_swept_col_name2 = 'total_swept_vol_' + 'STAGE II'
    swept_HEvol_col_name2 = 'swept_vol_HE_' + 'STAGE II'
    swept_CEvol_col_name2 = 'swept_vol_CE_' + 'STAGE II'
    polymetric_expo_n_col_name2 = 'polymetric_expo_n_' + 'STAGE II'
    HE_clearance_vol_Vs_col_name2 = 'HE_clearance_vol_Vs_' + 'STAGE II'
    V4_TDC_col_name2='(V4) TDC'+'STAGE II'
    head_end_V1_col_name2='head_end_V1'+'STAGE II'
    BDC_V2_col_name2='BDC_V2'+'STAGE II'
    HE_V3_col_name2='HE_V3'+'STAGE II'
    CE_clearance_vol_Vs_col_name2='CE_clearance_vol_Vs'+'STAGE II'
    TDC_V4_col_name2='TDC_V4'+'STAGE II'
    CE_V1_col_name2='CE_V1'+'STAGE II'
    BDC_V2_col_name22='BDC_V2_'+'STAGE II'
    HE_V3_col_name22='HE_V3(2)'+'STAGE II'
    HE_inlet_vol_eff_col_name2='HE_inlet_vol_eff %'+'STAGE II'
    CE_inlet_vol_eff_col_name2='CE_inlet_vol_eff %'+'STAGE II'
    HE_discharge_vol_eff_col_name2='HE_discharge_vol_eff %'+'STAGE II'
    CE_discharge_vol_eff_col_name2='CE_discharge_vol_eff %'+'STAGE II'
    cal_avg_inlet_vol_eff_col_name2='cal_avg_inlet_vol_eff%'+'STAGE II'
    design_avg_inlet_vol_eff_col_name2='design_avg_inlet_vol_eff%'+'STAGE II'
    overall_vol_eff_col_name2='overall_vol_eff%'+'STAGE II'
    stage_inlet_capacity_Q_col_name2='stage_inlet_capacity_Q'+'STAGE II'
    stage_discharge_capacity_Q_col_name2='stage_discharge_capacity_Q'+'STAGE II'
    actual_disc_temp_S2_col_name2='actual_disc_temp_S2'+'STAGE II'
    rod_load_tension_col_name2='rod_load_tension'+'STAGE II'
    rod_load_compression_col_name2='rod_load_compression'+'STAGE II'
    max_allowed_gas_load_col_name2='max_allowed_gas_load'+'STAGE II'
    Zs_S2_col_name2='Zs_S2'+'STAGE II'
    Zd_S2_col_name2='Zd_S2'+'STAGE II'
    Zavg_col_name2='Zavg'+'STAGE II'
    Vsp_inlet_S2_col_name2='Vsp_inlet_S2'+'STAGE II'
    Vsp_discharge_S2_col_name2='Vsp_discharge_S2'+'STAGE II'
    stage_inlet_capacity_M_col_name2='stage_inlet_capacity_M'+'STAGE II'
    stage_discharge_capacity_M_col_name2='stage_discharge_capacity_M'+'STAGE II'
    k_col_name2='k'+'STAGE II'
    k_k1_col_name2='k/k-1'+'STAGE II'
    Had_col_name2='Had'+'STAGE II'
    Adiabatic_disc_temp_col_name2='Adiabatic_disc_temp'+'STAGE II'
    Adiabatic_eff_col_name2='Adiabatic_eff%'+'STAGE II'
    Pad_col_name2='Pad'+'STAGE II'
    Zd_adiabatic_temp_disc_press_col_name2='Zd_adiabatic_temp_disc_press'+'STAGE II'
    Vsp_discharge_col_name2='Vsp_discharge'+'STAGE II'
    adiabatic_flow_factor_col_name2='adiabatic_flow_factor'+'STAGE II'
    flow_factor_S2_col_name2='flow_factor_S2'+'STAGE II'
    disc_temp_ad_disc_temp_col_name2='disc_temp-ad_disc_temp'+'STAGE II'
    
    df2_proc_data[suction_col_name2] = df2_proc_data['S2_inlet_pressure'] + (df1_config.loc[df1_config['tag_name'] == 'atmospheric_pressure', 'stg2_values'].iloc[0])
    df2_proc_data[discharge_col_name2] = (df1_config.loc[df1_config['tag_name'] == 'dp_flange_discharge_line', 'stg2_values'].iloc[0]) + (df1_config.loc[df1_config['tag_name'] == 'atmospheric_pressure', 'stg2_values'].iloc[0])+df2_proc_data['S2_discharge_pressure']
    df2_proc_data[ratio_col_name2] = df2_proc_data[discharge_col_name2] / df2_proc_data[suction_col_name2]
    df2_proc_data[tot_swept_col_name2] = df1_config.loc[df1_config['tag_name'] == 'total_swept_volume', 'stg2_values'].iloc[0]
    df2_proc_data[swept_HEvol_col_name2] = df1_config.loc[df1_config['tag_name'] == 'swept_volume_head_end', 'stg2_values'].iloc[0]
    df2_proc_data[swept_CEvol_col_name2] = df1_config.loc[df1_config['tag_name'] == 'swept_volume_crank_end', 'stg2_values'].iloc[0]
    df2_proc_data[HE_clearance_vol_Vs_col_name2] = ((df1_config.loc[df1_config['tag_name'] == 'head_end_clearance', 'stg2_values'].iloc[0])/100)*df2_proc_data[swept_HEvol_col_name2]
    df2_proc_data[V4_TDC_col_name2]=df2_proc_data[HE_clearance_vol_Vs_col_name2]
    df2_proc_data[polymetric_expo_n_col_name2] = 1 / (1 - ((np.log((df2_proc_data['S2_discharge_temp'] + 273) / (df2_proc_data['S2_suction_srubber_out'] + 273))) / (np.log((df2_proc_data[discharge_col_name2]/df2_proc_data[suction_col_name2]))))) 
    df2_proc_data[head_end_V1_col_name2]=df2_proc_data[V4_TDC_col_name2]*((df2_proc_data[ratio_col_name2])**(1/df2_proc_data[polymetric_expo_n_col_name2]))
    df2_proc_data[BDC_V2_col_name2]=df2_proc_data[swept_HEvol_col_name2]+df2_proc_data[HE_clearance_vol_Vs_col_name2]
    df2_proc_data[HE_V3_col_name2]=df2_proc_data[BDC_V2_col_name2]*((1/(df2_proc_data[ratio_col_name2]))**(1/df2_proc_data[polymetric_expo_n_col_name2]))
    df2_proc_data[CE_clearance_vol_Vs_col_name2]=((df1_config.loc[df1_config['tag_name'] == 'crank_end_clearance', 'stg2_values'].iloc[0])/100)*df2_proc_data[swept_CEvol_col_name2]
    df2_proc_data[TDC_V4_col_name2]=df2_proc_data[CE_clearance_vol_Vs_col_name2]
    df2_proc_data[CE_V1_col_name2]=df2_proc_data[CE_clearance_vol_Vs_col_name2]*((df2_proc_data[ratio_col_name2])**(1/df2_proc_data[polymetric_expo_n_col_name2]))
    df2_proc_data[BDC_V2_col_name22]=df2_proc_data[swept_CEvol_col_name2]+df2_proc_data[CE_clearance_vol_Vs_col_name2]
    df2_proc_data[HE_V3_col_name22]=df2_proc_data[BDC_V2_col_name22]*((1/(df2_proc_data[ratio_col_name2]))**(1/df2_proc_data[polymetric_expo_n_col_name2]))
    df2_proc_data[HE_inlet_vol_eff_col_name2]=((df2_proc_data[BDC_V2_col_name2]-df2_proc_data[head_end_V1_col_name2])*100)/(df2_proc_data[BDC_V2_col_name2]-df2_proc_data[V4_TDC_col_name2])
    df2_proc_data[CE_inlet_vol_eff_col_name2]=((df2_proc_data[BDC_V2_col_name22]-df2_proc_data[CE_V1_col_name2])*100)/(df2_proc_data[BDC_V2_col_name22]-df2_proc_data[TDC_V4_col_name2])
    df2_proc_data[HE_discharge_vol_eff_col_name2]=((df2_proc_data[HE_V3_col_name2]-df2_proc_data[V4_TDC_col_name2])*100)/(df2_proc_data[BDC_V2_col_name2]-df2_proc_data[V4_TDC_col_name2])
    df2_proc_data[CE_discharge_vol_eff_col_name2]=(((df2_proc_data[HE_V3_col_name22]-df2_proc_data[TDC_V4_col_name2]))*100)/(df2_proc_data[BDC_V2_col_name22]-df2_proc_data[TDC_V4_col_name2])
    df2_proc_data[cal_avg_inlet_vol_eff_col_name2]=(df2_proc_data[HE_inlet_vol_eff_col_name2]+df2_proc_data[CE_inlet_vol_eff_col_name2])/2
    df2_proc_data[design_avg_inlet_vol_eff_col_name2]=(df1_config.loc[df1_config['tag_name'] == 'volumetric_efficiency', 'stg2_values'].iloc[0])
    df2_proc_data[overall_vol_eff_col_name2]=(df2_proc_data[HE_inlet_vol_eff_col_name2]+df2_proc_data[CE_inlet_vol_eff_col_name2]+df2_proc_data[HE_discharge_vol_eff_col_name2]+df2_proc_data[CE_discharge_vol_eff_col_name2])/4
    df2_proc_data[stage_inlet_capacity_Q_col_name2]=df2_proc_data[cal_avg_inlet_vol_eff_col_name2]*df2_proc_data[tot_swept_col_name2]/100
    df2_proc_data[stage_discharge_capacity_Q_col_name2]=(df2_proc_data[HE_discharge_vol_eff_col_name2]+df2_proc_data[CE_discharge_vol_eff_col_name2])/2*df2_proc_data[tot_swept_col_name2]/100
    df2_proc_data[actual_disc_temp_S2_col_name2]=df2_proc_data['S2_discharge_temp'] + 273
    df2_proc_data[rod_load_tension_col_name2]=(((df1_config.loc[df1_config['tag_name'] == 'cross_sectional_area_of_piston', 'stg2_values'].iloc[0]) * (df2_proc_data[discharge_col_name2] - df2_proc_data[suction_col_name2])) - ((df1_config.loc[df1_config['tag_name'] == 'cross_sectional_area_of_piston_rod', 'stg2_values'].iloc[0]) * df2_proc_data[discharge_col_name2])) * 10**5
    df2_proc_data[rod_load_compression_col_name2]=(((df1_config.loc[df1_config['tag_name'] == 'cross_sectional_area_of_piston', 'stg2_values'].iloc[0]) * (df2_proc_data[discharge_col_name2] - df2_proc_data[suction_col_name2])) + ((df1_config.loc[df1_config['tag_name'] == 'cross_sectional_area_of_piston_rod', 'stg2_values'].iloc[0]) * df2_proc_data[suction_col_name2])) * 10**5
    df2_proc_data[max_allowed_gas_load_col_name2]=df1_config.loc[df1_config['tag_name'] == 'max_allowed_gas_rod_load', 'stg2_values'].iloc[0]
    df2_proc_data[Zs_S2_col_name2]=df_coeff_summary.iloc[9, 1] + (df_coeff_summary.iloc[9, 3] * df2_proc_data['S2_suction_srubber_out'])+ (df_coeff_summary.iloc[9, 4] * df2_proc_data[suction_col_name2] * 100) + (df_coeff_summary.iloc[9, 5]* ((df2_proc_data['S2_suction_srubber_out'] ** 2))) + (df_coeff_summary.iloc[9, 6] * df2_proc_data['S2_suction_srubber_out'] * df2_proc_data[suction_col_name2] * 100) + (df_coeff_summary.iloc[9, 7] * ((df2_proc_data[suction_col_name2] * 100) ** 2))
    df2_proc_data[Zd_S2_col_name2]=df_coeff_summary.iloc[7, 1] + (df_coeff_summary.iloc[7, 3] * df2_proc_data['S2_discharge_temp'])+ (df_coeff_summary.iloc[7, 4] * df2_proc_data[discharge_col_name2] * 100) + (df_coeff_summary.iloc[7, 5]* ((df2_proc_data['S2_discharge_temp'] ** 2))) + (df_coeff_summary.iloc[7, 6] * df2_proc_data['S2_discharge_temp'] * df2_proc_data[discharge_col_name2] * 100) + (df_coeff_summary.iloc[7, 7] * ((df2_proc_data[discharge_col_name2] * 100) ** 2))
    df2_proc_data[Zavg_col_name2]=(df2_proc_data[Zs_S2_col_name2]+df2_proc_data[Zd_S2_col_name2])/2
    df2_proc_data[Vsp_inlet_S2_col_name2]=df2_proc_data[Zs_S2_col_name2]*vsp_const*(df2_proc_data['S2_suction_srubber_out'] + 273)/((df1_config.loc[df1_config['tag_name'] == 'mol_wt', 'stg2_values'].iloc[0])*df2_proc_data[suction_col_name2]*(10**5))
    df2_proc_data[Vsp_discharge_S2_col_name2]=df2_proc_data[Zd_S2_col_name2]*vsp_const*(df2_proc_data['S2_discharge_temp'] + 273)/((df1_config.loc[df1_config['tag_name'] == 'mol_wt', 'stg2_values'].iloc[0])*df2_proc_data[discharge_col_name2]*(10**5))
    df2_proc_data[stage_inlet_capacity_M_col_name2]=df2_proc_data[stage_inlet_capacity_Q_col_name2]/df2_proc_data[Vsp_inlet_S2_col_name2]
    df2_proc_data[stage_discharge_capacity_M_col_name2]=df2_proc_data[stage_discharge_capacity_Q_col_name2]/df2_proc_data[Vsp_discharge_S2_col_name2]
    df2_proc_data[k_col_name2]=((df_coeff_summary.iloc[3, 1] + (df_coeff_summary.iloc[3, 3] * df2_proc_data['S2_suction_srubber_out'])+ (df_coeff_summary.iloc[3, 4] * df2_proc_data[suction_col_name2] * 100) + (df_coeff_summary.iloc[3, 5]* ((df2_proc_data['S2_suction_srubber_out'] ** 2))) + (df_coeff_summary.iloc[3, 6] * df2_proc_data['S2_suction_srubber_out'] * df2_proc_data[suction_col_name2] * 100) + (df_coeff_summary.iloc[3, 7] * ((df2_proc_data[suction_col_name2] * 100) ** 2)))+(df_coeff_summary.iloc[1, 1] + (df_coeff_summary.iloc[1, 3] * df2_proc_data['S2_discharge_temp'])+ (df_coeff_summary.iloc[1, 4] * df2_proc_data[discharge_col_name2] * 100) + (df_coeff_summary.iloc[1, 5]* ((df2_proc_data['S2_discharge_temp'] ** 2))) + (df_coeff_summary.iloc[1, 6] * df2_proc_data['S2_discharge_temp'] * df2_proc_data[discharge_col_name2] * 100) + (df_coeff_summary.iloc[1, 7] * ((df2_proc_data[discharge_col_name2] * 100) ** 2))))/2
    df2_proc_data[k_k1_col_name2]=df2_proc_data[k_col_name2]/(df2_proc_data[k_col_name2]-1)
    df2_proc_data[Had_col_name2]=df2_proc_data[Zavg_col_name2]*(vsp_const/(df1_config.loc[df1_config['tag_name'] == 'mol_wt', 'stg2_values'].iloc[0]))*(df2_proc_data['S2_suction_srubber_out'] + 273)*df2_proc_data[k_k1_col_name2]*(df2_proc_data[ratio_col_name2]**(1/df2_proc_data[k_k1_col_name2])-1)
    df2_proc_data[Adiabatic_disc_temp_col_name2]=(df2_proc_data['S2_suction_srubber_out'] + 273)*(df2_proc_data[ratio_col_name2]**(1/df2_proc_data[k_k1_col_name2]))
    df2_proc_data[Adiabatic_eff_col_name2]=(df2_proc_data[Adiabatic_disc_temp_col_name2]-((df2_proc_data['S2_suction_srubber_out'] + 273)))*100/((df2_proc_data['S2_discharge_temp'])-(df2_proc_data['S2_suction_srubber_out']))
    df2_proc_data[Pad_col_name2]=df2_proc_data[stage_inlet_capacity_M_col_name2]*df2_proc_data[Had_col_name2]/(df2_proc_data[Adiabatic_eff_col_name2]/100)*(1/time_conv)
    df2_proc_data[Zd_adiabatic_temp_disc_press_col_name2]=df_coeff_summary.iloc[7, 1] + (df_coeff_summary.iloc[7, 3] * (df2_proc_data[Adiabatic_disc_temp_col_name2]-273))+ (df_coeff_summary.iloc[7, 4] * df2_proc_data[discharge_col_name2] * 100) + (df_coeff_summary.iloc[7, 5]* (((df2_proc_data[Adiabatic_disc_temp_col_name2]-273) ** 2))) + (df_coeff_summary.iloc[7, 6] * (df2_proc_data[Adiabatic_disc_temp_col_name2]-273) * df2_proc_data[discharge_col_name2] * 100) + (df_coeff_summary.iloc[7, 7] * ((df2_proc_data[discharge_col_name2] * 100) ** 2))
    df2_proc_data[Vsp_discharge_col_name2]=df2_proc_data[Zd_adiabatic_temp_disc_press_col_name2]*(vsp_const/(df1_config.loc[df1_config['tag_name'] == 'mol_wt', 'stg2_values'].iloc[0]))*(df2_proc_data[Adiabatic_disc_temp_col_name2]/(df2_proc_data[suction_col_name2]*(10**5)))
    df2_proc_data[adiabatic_flow_factor_col_name2]=df2_proc_data[stage_inlet_capacity_M_col_name2]/(df2_proc_data[stage_inlet_capacity_Q_col_name2]/df2_proc_data[Vsp_discharge_col_name2])
    df2_proc_data[flow_factor_S2_col_name2]=df2_proc_data[stage_inlet_capacity_M_col_name2]/df2_proc_data[stage_discharge_capacity_M_col_name2]

calc(df1_config, df2_proc_data, stage) 

print(df2_proc_data.head())

        Date      Time  S1_suction_temp  S1_discharge_temp  HE_outlet  \
0 2019-01-14  18:00:00            23.64              51.70      21.55   
1 2019-01-14  19:00:00            23.96              55.85      21.20   
2 2019-01-14  20:00:00            24.29              59.99      20.85   
3 2019-01-14  21:00:00            24.62              64.14      20.50   
4 2019-01-14  22:00:00            24.95              68.29      20.15   

   S2_suction_srubber_out  S2_discharge_temp  inlet_to_oilfilter  \
0                   21.82              51.23               49.82   
1                   21.40              56.07               53.12   
2                   20.99              60.91               56.41   
3                   20.57              65.75               59.71   
4                   20.15              70.59               63.00   

   compressor_main_bearingrtd  compressor_main_bearingrtd  ...  kSTAGE II  \
0                       52.03                       51.76  ...   1.390185  

In [234]:
#temperature_value = df_coeff_summary.loc[('Cp/Cv', 'Suction_1'), 'Intercept']
#value = df_coeff_summary.loc[('Cp/Cv', 2)]
#x=df_coeff_summary.loc[('Cp/Cv', value)]
#print(x)
#print(temperature_value)

In [235]:
print(df_coeff_summary.index.tolist())


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


In [236]:
print(df1_config.head())

                             tag_name  stg1_values  stg2_values
0                  reference_pressure      1.01325          NaN
1               reference_temperature     15.60000          NaN
2                 suction_temperature     54.40000     55.00000
3  stg_inlet_capacity_volumetric_flow   2834.90000  34143.20689
4                        cylinder_dia    330.20000    241.30000
